# SQL on BigQuery

```
SELECT limit_balance, sex, education_level, marital_status, age, 
       pay_0, pay_2, pay_3, pay_4, pay_5, pay_6, 
       bill_amt_1, bill_amt_2, bill_amt_3, bill_amt_4, bill_amt_5, bill_amt_6, 
       pay_amt_1, pay_amt_2, pay_amt_3, pay_amt_4, pay_amt_5, pay_amt_6,
       default_payment_next_month
FROM `bigquery-public-data.ml_datasets.credit_card_default`
LIMIT 37981
```

# 1 - Perkenalan
  > Bab pengenalan harus diisi dengan identitas, gambaran besar dataset yang digunakan, dan _objective_ yang ingin dicapai.

## 1 - Identity

  Nama : Jason Rich Darmawan Onggo Putra
  
  Batch : 016 RMT

## 2 - Dataset Overview

## 3 - Objective

  > Buatlah model Classification untuk memprediksi default_payment_next_month menggunakan dataset yang sudah kalian simpan.

  Choose an algorithm with default parameters that have the best metrics. Then, improve the algorithm with tuned parameters.

  The algorithm should have metircs:
  1. accuracy score of 0.8
  2. recall score of 0.9 for class 1
  3. precision score of 0.9 for class 1
  4. f1 score of 0.9 for class 1.

# 2 - Import Libraries
  > _Cell_ pertama pada _notebook_ **harus berisi dan hanya berisi** semua _library_ yang digunakan dalam _project_.

In [1]:
# Data Loading
import pandas as pd

# 3 - Data Loading
  > Bagian ini berisi proses penyiapan data sebelum dilakukan eksplorasi data lebih lanjut. Proses Data Loading dapat berupa memberi nama baru untuk setiap kolom, mengecek ukuran dataset, dll.

In [2]:
# Load Dataset
dataset = pd.read_csv("./h8dsft_P1M1_jason_rich_darmawan_onggo_putra.csv")

## 1 - Columns Data Types

X or features:

- numerical data: 
  - 'limit_balance', 'age'
  - 'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6'
  - 'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6'
  - 'pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6' [outline](#41---what-are-pay_0-bill_amt_1-pay_amt_1)
- nominal data: 'sex', 'education_level', 'marital_status'

y or target:

- ordinal data: 'default_payment_next_month'

In [3]:
# head and tail
dataset

,limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,pay_5,...,bill_amt_4,bill_amt_5,bill_amt_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,default_payment_next_month
0,50000,1,3,0,50,0,0,0,-1,-1,...,3400,2769,0,2266,0,3400,2769,1700,0,0
1,200000,2,3,0,45,-2,-2,-2,-2,-2,...,399,1396,0,2430,0,399,1396,0,0,0
2,130000,1,1,1,45,0,0,0,0,0,...,62377,63832,65099,2886,2908,2129,2354,2366,2291,0
3,230000,1,2,1,41,0,0,0,0,0,...,216260,215968,174195,9405,7769,7757,8030,6549,5226,1
4,50000,1,2,1,42,0,0,0,0,0,...,31320,36227,35569,2007,2000,1500,5400,0,3000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2960,320000,1,2,1,41,1,-2,-2,-2,-2,...,0,700,860,0,0,0,700,860,2420,1
2961,20000,1,1,1,39,-1,-1,-1,-1,-1,...,630,664,598,700,600,700,700,600,1400,0
2962,20000,1,2,2,25,0,0,0,0,0,...,16600,17025,17381,1300,1300,1300,700,630,650,0
2963,50000,1,2,2,38,0,0,0,0,0,...,17833,18246,18642,1300,1600,650,700,700,700,0


## 2 - Columns Dtypes

- no missing values
- [x] change columns dtypes to 'object': 'sex', 'education_level', 'marital_status'

- numerical data: 
  - 'limit_balance', 'age'
  - 'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6'
  - 'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6'
  - 'pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6' [outline](#41---what-are-pay_0-bill_amt_1-pay_amt_1)
- nominal data: 
  - 'sex'
  - 'education_level'
    
    description:
    > Education Level (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
  - 'marital_status'
    
    description:
    > Marital status (1=married, 2=single, 3=others)


In [4]:
# find the columns dtypes
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2965 entries, 0 to 2964
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   limit_balance               2965 non-null   int64
 1   sex                         2965 non-null   int64
 2   education_level             2965 non-null   int64
 3   marital_status              2965 non-null   int64
 4   age                         2965 non-null   int64
 5   pay_0                       2965 non-null   int64
 6   pay_2                       2965 non-null   int64
 7   pay_3                       2965 non-null   int64
 8   pay_4                       2965 non-null   int64
 9   pay_5                       2965 non-null   int64
 10  pay_6                       2965 non-null   int64
 11  bill_amt_1                  2965 non-null   int64
 12  bill_amt_2                  2965 non-null   int64
 13  bill_amt_3                  2965 non-null   int64
 14  bill_amt

In [56]:
dataset[['sex', 'education_level', 'marital_status']] = \
    dataset[['sex', 'education_level', 'marital_status']].astype('object')

# 4 - Exploratory Data Analysis (EDA)
  > Bagian ini berisi eksplorasi data pada dataset diatas dengan menggunakan query, grouping, visualisasi sederhana, dan lain sebagainya.

  Task:
  1. insight

## 4.1 - what are 'pay_0', 'bill_amt_1', 'pay_amt_1'

- [x] why 'pay_0' have value '-2'

  EDA shows there are '-2'. We assume this as:
  1. '-2'=payment 2 months in advance
  2. '-1'=payment 1 months in advance
  3. '0'=pay duly. (label with the most frequency)
  4. '1'=payment delay for one month

- [x] is 'bill_amt_1' paid in 'pay_amt_1'?

  EDA shows there are a handful of people that paid in 'pay_amt_1' according to 'bill_amt_1'. Therefore, we assume that 'bill_amt_1' is billing statement of the month and paid in 'pay_amt_1'.

  > [Billing statements are issued monthly at the end of each billing cycle.](https://www.investopedia.com/terms/b/billing-statement.asp)

description:
- 'pay_0' 
  > Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)

- 'pay_2'
  > Repayment status in August, 2005 (scale same as above)

- 'bill_amt_1'
  > Amount of bill statement in September, 2005 (NT dollar)

- 'bill_amt_2'
  > Amount of bill statement in August, 2005 (NT dollar)

- 'pay_amt_1'
  > Amount of previous payment in September, 2005 (NT dollar)

- 'pay_amt_2' 
  > Amount of previous payment in August, 2005 (NT dollar)

In [55]:
# is bill_amt_1 paid in pay_amt_1 ?
dataset.loc[(dataset['bill_amt_1'] == dataset['pay_amt_1']) &
            (dataset['bill_amt_2'] == dataset['pay_amt_2']) &
            (dataset['bill_amt_1'] != 0), 
            ['pay_0', 'pay_2', 'pay_3',
             'bill_amt_1', 'bill_amt_2', 'bill_amt_3',
             'pay_amt_1', 'pay_amt_2', 'pay_amt_3']][:10]

,pay_0,pay_2,pay_3,bill_amt_1,bill_amt_2,bill_amt_3,pay_amt_1,pay_amt_2,pay_amt_3
567,-1,-1,-1,396,396,396,396,396,396
579,-1,-1,-1,415,415,415,415,415,565
593,2,-1,-1,326,326,326,326,326,480
599,-1,-1,-1,223,223,223,223,223,223
611,-1,-1,-1,316,316,316,316,316,316
624,-1,-1,-1,552,552,552,552,552,552
630,-1,-1,-1,396,396,396,396,396,396
660,-1,-1,-1,396,396,396,396,396,2707
671,-1,-1,-1,860,860,860,860,860,860
673,-1,-1,-1,316,316,316,316,316,0


In [43]:
# check the data distribution
dataset['pay_0'].value_counts().sort_index(axis=0)

-2     247
-1     540
 0    1516
 1     356
 2     262
 3      29
 4       7
 5       2
 6       1
 7       2
 8       3
Name: pay_0, dtype: int64

In [45]:
# check the min max of each features.
dataset[['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']].agg(['min', 'max'])

,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6
min,-2,-2,-2,-2,-2,-2
max,8,7,7,8,7,7


## 2 - Relationship between X Numerical and y Categorical

X or features:

- numerical data: 
  - 'limit_balance', 'age'
  - 'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6'
  - 'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6'
  - 'pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6' [outline](#41---what-are-pay_0-bill_amt_1-pay_amt_1)
- nominal data: 'sex', 'education_level', 'marital_status'

y or target:

- ordinal data: 'default_payment_next_month'

# 5 - Data Preprocessing
  > Bagian ini berisi proses penyiapan data untuk proses pelatihan model, seperti pembagian data menjadi train-dev-test, transformasi data (normalisasi, encoding, dll.), dan proses-proses lain yang dibutuhkan.

# 6 - Model Definition
  > Bagian ini berisi cell untuk mendefinisikan model. Jelaskan alasan menggunakan suatu algoritma/model, hyperparameter yang dipakai, jenis penggunaan metrics yang dipakai, dan hal lain yang terkait dengan model.

## 1 - Logistic Regression

## 2 - SVM

## 3 - Decision Tree

## 4 - KKNN

## 5 - Naive Bayes

## 6 - Other Algorithm

# 7 - Model Training
  > Cell pada bagian ini hanya berisi code untuk melatih model dan output yang dihasilkan. Lakukan beberapa kali proses training dengan hyperparameter yang berbeda untuk melihat hasil yang didapatkan. Analisis dan narasikan hasil ini pada bagian Model Evaluation.

## 1 - Default Parameter

### 1 - Logistic Regression

### 2 - SVM

### 3 - Decision Tree

### 4 - KKNN

### 5 - Naive Bayes

### 6 - Other Algorithm

## 2 - Model Tuning

  Goal: To find the best parameters for the algorithm.

### 1 - `sklearn.model_selection.GridSearchCV` (based on best algorithm with default parameters)

  Goal: To find the best parameters for the algorithm.

  Task: 
  1. compare the metrics between algorithm with default parameters and best parameters.

#### 1 - Logistic Regression

#### 2 - SVM

#### 3 - Decision Tree

#### 4 - KKNN

#### 5 - Naive Bayes

#### 6 - Other Algorithm

# 8 - Model Evaluation
  > Pada bagian ini, dilakukan evaluasi model yang harus menunjukkan bagaimana performa model berdasarkan metrics yang dipilih. Hal ini harus dibuktikan dengan visualisasi tren performa dan/atau tingkat kesalahan model. **Lakukan analisis terkait dengan hasil pada model dan tuliskan hasil analisisnya**.

## 1 - `sklearn.model_selection.cross_validate` and `sklearn.metrics`

  Goal: To validate if the metrics aree consistent and not due to data leakage.

  Metrics used are: `sklearn.metrics.precision_score`, `sklearn.metrics.recall_score`, `sklearn.metrics.f1_score` with parameter 'average=None' and `sklearn.metrics.accuracy_score`.

  Task:
  1. metrics interpretation.
  2. weakness/strength of the model.
  3. improvement.

### 1 - Default Parameters

#### 1 - Logistic Regression

#### 2 - SVM

#### 3 - Decision Tree

#### 4 - KKNN

#### 5 - Naive Bayes

#### 6 - Other Algorithm

### 2 - (Algorithm) with Tuned Parameters

  Task:
  1. Only best algorithm gets evaluated.

# 8 - Model Inference
  > Model yang sudah dilatih akan dicoba pada data yang bukan termasuk ke dalam train-set ataupun test-set. Data ini harus dalam format yang asli, bukan data yang sudah di-scaled.

## 1 - (Algorithm) with Tuned Parameters

  Task:
  1. Only best algorithm gets deployed.

# 10 - Pengambilan Kesimpulan
  > Pada bagian terakhir ini, **harus berisi** kesimpulan yang mencerminkan hasil yang didapat dengan _objective_ yang sudah ditulis di bagian pengenalan.

# Conceptual Problems

1. Apakah fungsi parameter `criterion` pada Decision Tree? Jelaskan salah satu criterion yang kalian pahami!
2. Apakah fungsi dari `pruning` pada Tree model?
3. Bagaimana cara memilih `K` yang optimal pada KNN?
4. Jelaskan apa yang kalian ketahui tentang `Cross Validation`!
5. Jelaskan apa yang kalian ketahui tentang `Accuracy, Precision, Recall, F1 Score`!